In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Đường dẫn file CSV để lưu dataset
chrome_options = Options()
chrome_options.add_argument("--lang=vi")

browser = webdriver.Chrome(options=chrome_options)
# Link địa điểm Google Maps
links = [
    "https://www.google.com/maps/search/Cao+B%E1%BA%B1ng+B%E1%BB%87nh+Vi%E1%BB%87n/@22.6548202,106.2646583,17z/data=!3m1!4b1?entry=ttu&g_ep=EgoyMDI1MDUyMS4wIKXMDSoASAFQAw%3D%3D"
]
# Khởi tạo trình duyệt
browser = webdriver.Chrome()
record = []  # Danh sách để lưu dữ liệu

def Selenium_extractor(link, filename):
    action = ActionChains(browser)
    record = []  # Danh sách để lưu dữ liệu

    # Truy cập Google Maps (bạn đã mở trang này thủ công trong trình duyệt)
    browser.get(link)

    # Đợi trang tải hoàn chỉnh
    WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "hfpxzc")))

    # Đợi đến khi các địa điểm được tải xong
    WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "hfpxzc")))

    # Lấy danh sách các địa điểm ban đầu
    locations = browser.find_elements(By.CLASS_NAME, "hfpxzc")

    # Cuộn để tải thêm địa điểm
    while len(locations) < 10000:
        print("Số lượng địa điểm:", len(locations))
        var = len(locations)
        scroll_origin = ScrollOrigin.from_element(locations[-1])
        action.scroll_from_origin(scroll_origin, 0, 1000).perform()
        time.sleep(2)
        locations = browser.find_elements(By.CLASS_NAME, "hfpxzc")
        if len(locations) == var:  # Nếu không tải thêm địa điểm, thoát
            break


    # Lặp qua từng địa điểm để lấy đánh giá
    for i in range(len(locations)):
        try:
            # Cuộn đến địa điểm và click
            scroll_origin = ScrollOrigin.from_element(locations[i])
            action.scroll_from_origin(scroll_origin, 0, 100000).perform()
            action.move_to_element(locations[i]).perform()
            browser.execute_script("arguments[0].click();", locations[i])

            # Đợi trang tải và lấy mã HTML
            WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "DUwDvf")))

            # Nhấn nút "Bài đánh giá khác"
            more_reviews_button = WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Bài đánh giá khác')]"))
            )
            more_reviews_button.click()
            time.sleep(5)  # Đợi một chút để các đánh giá mới tải

            # Chuyển đến tab chứa phần đánh giá
            review_tab = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "m6QErb"))
            )

            # Cuộn tab chứa phần đánh giá
            for _ in range(5):  # Cuộn 5 lần (có thể điều chỉnh tùy thuộc vào nhu cầu)
                browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_tab)
                time.sleep(2)  # Đợi một chút sau mỗi lần cuộn

            # Lấy HTML của trang và tách với BeautifulSoup
            source = browser.page_source
            soup = BeautifulSoup(source, 'html.parser')

            # Tên bệnh viện
            name_html = soup.find('span', {"class": "iD2gKb WlneJ"})
            hospital_name = name_html.text.strip() if name_html else "Unknown"

            # Lấy rating của địa điểm
            rating_html = soup.find('div', {"class": "FontDisplayLarge"})
            rating = rating_html.text.strip() if rating_html else "Chưa có đánh giá"

            # Lấy tất cả các đánh giá và sao của mỗi bình luận
            reviews_html = soup.find_all('span', {"class": "wiI7pd"})
            star_ratings_html = soup.find_all('span', {"class": "kvMYJc"}, attrs={"aria-label": True})  # Lấy sao

            # Kiểm tra số lượng đánh giá và sao có khớp không
            for review, star_rating in zip(reviews_html, star_ratings_html):
                comment = review.text.strip()
                star = star_rating['aria-label'] if 'aria-label' in star_rating.attrs else "Không có sao"

                # Kiểm tra xem bình luận có phải bằng tiếng Việt hay không (lọc bình luận)
                if any(ord(c) > 127 for c in comment):  # Kiểm tra nếu bình luận có ký tự tiếng Việt
                    # Thêm vào danh sách dữ liệu
                    record.append((comment, hospital_name, star, rating))

            # Lưu dữ liệu vào file CSV sau mỗi lượt quét
            df = pd.DataFrame(record, columns=["comment", "hospital", "star", "rating"])
            df.to_csv(filename, index=False, encoding='utf-8-sig')

        except Exception as e:
            print(f"Lỗi khi xử lý địa điểm {i}: {e}")
            continue


# Lặp qua các liên kết và thu thập dữ liệu từ mỗi liên kết
for i, link in enumerate(links):
    filename = f"C:/Noname/NLP/data/hospital_reviews_{i+2}.csv"  # Tạo tên file CSV duy nhất cho mỗi link
    Selenium_extractor(link, filename)

# Đóng trình duyệt
browser.quit()

print(f"Dữ liệu được lưu vào {filename}")
